# **Red Neuronal Predictora para la Copa del Mundo 2026** #

## **Importar librerías y cargar datos** ##

In [6]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.layers import Input, Embedding, Flatten, Concatenate, Dense, Dropout
from keras.models import Model

print(f"Usando TensorFlow versión: {tf.__version__}")

url = 'https://github.com/martj42/international_results/raw/master/results.csv'
df = pd.read_csv(url)

Usando TensorFlow versión: 2.20.0


## **Limpieza y tratamiento de datos** ##

In [7]:
df['date'] = pd.to_datetime(df['date'])
df = df[df['date'].dt.year >= 1990].reset_index(drop=True)

def get_result(row):
    if row['home_score'] > row['away_score']:
        return 2
    elif row['home_score'] < row['away_score']:
        return 0
    else:
        return 1

df['result'] = df.apply(get_result, axis=1)

## **Codificar los equipos y preparar los datos** ##

In [8]:
all_teams = pd.concat([df['home_team'], df['away_team']]).unique()
team_encoder = LabelEncoder().fit(all_teams)

df['home_team_encoded'] = team_encoder.transform(df['home_team'])
df['away_team_encoded'] = team_encoder.transform(df['away_team'])

lista_de_equipos_conocidos = sorted(list(team_encoder.classes_))
print(f"El modelo ha sido entrenado con {len(lista_de_equipos_conocidos)} equipos en total.")
print("Asegúrate de que los nombres en tu simulación coincidan con esta lista.")

X = df[['home_team_encoded', 'away_team_encoded']]
y = df['result']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

El modelo ha sido entrenado con 323 equipos en total.
Asegúrate de que los nombres en tu simulación coincidan con esta lista.


## **Construir el modelo con Embedding** ##

In [9]:
n_teams = len(team_encoder.classes_)
embedding_dim = 10

input_home = Input(shape=(1,), name='Entrada_Equipo_Local')
input_away = Input(shape=(1,), name='Entrada_Equipo_Visitante')

embedding_layer = Embedding(input_dim=n_teams, output_dim=embedding_dim, name='Capa_Embedding')

embedded_home = Flatten()(embedding_layer(input_home))
embedded_away = Flatten()(embedding_layer(input_away))

concat_layer = Concatenate()([embedded_home, embedded_away])

dense1 = Dense(128, activation='relu')(concat_layer)
dropout1 = Dropout(0.5)(dense1)
dense2 = Dense(64, activation='relu')(dropout1)
output = Dense(3, activation='softmax', name='Capa_Salida')(dense2)
model = Model(inputs=[input_home, input_away], outputs=output)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

print("\n--- Resumen de la Arquitectura del Modelo ---")
model.summary()


--- Resumen de la Arquitectura del Modelo ---


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ Entrada_Equipo_Loc… │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Entrada_Equipo_Vis… │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Capa_Embedding      │ (None, 1, 10)     │      3,230 │ Entrada_Equipo_L… │
│ (Embedding)         │                   │            │ Entrada_Equipo_V… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_2 (Flatten) │ (None, 10)        │          0 │ Capa_Embedding[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_3 (Flatten) │ (None, 10)        │          0 │ Capa_Embedding[1… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 20)        │          0 │ flatten_2[0][0],  │
│ (Concatenate)       │                   │            │ flatten_3[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 128)       │      2,688 │ concatenate_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 128)       │          0 │ dense_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 64)        │      8,256 │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Capa_Salida (Dense) │ (None, 3)         │        195 │ dense_4[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 14,369 (56.13 KB)

 Trainable params: 14,369 (56.13 KB)

 Non-trainable params: 0 (0.00 B)

## **Entrenar el modelo** ##

In [10]:
print("\n--- Iniciando Entrenamiento ---")
history = model.fit(
    [X_train['home_team_encoded'], X_train['away_team_encoded']],
    y_train,
    epochs=20,
    batch_size=32,
    validation_split=0.1,
    verbose=1
)
print("¡Entrenamiento completado!")


--- Iniciando Entrenamiento ---
Epoch 1/20
711/711 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.5413 - loss: 0.9663 - val_accuracy: 0.5901 - val_loss: 0.9046
Epoch 2/20
711/711 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5830 - loss: 0.9013 - val_accuracy: 0.5976 - val_loss: 0.8926
Epoch 3/20
711/711 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.5897 - loss: 0.8891 - val_accuracy: 0.5877 - val_loss: 0.8939
Epoch 4/20
711/711 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.5932 - loss: 0.8831 - val_accuracy: 0.5929 - val_loss: 0.8910
Epoch 5/20
711/711 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.5927 - loss: 0.8763 - val_accuracy: 0.5933 - val_loss: 0.8943
Epoch 6/20
711/711 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.5968 - loss: 0.8756 - val_accuracy: 0.5917 - val_loss: 0.8901
Epoch 7/20
711/711 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.5963 - loss: 0.8711 - val_accuracy: 0.5925 - val_loss: 0.8944
Epoch 8/20
711/711 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.5987 

## **Listado de paises para usar en la prediccion** ##

In [14]:
equipos_mundial_2026 = [
    'Canada', 'Mexico', 'United States',
    'Argentina', 'Brazil', 'Uruguay', 'Colombia', 'Ecuador', 'Peru',
    'Germany', 'Spain', 'France', 'England', 'Portugal', 'Netherlands',
    'Belgium', 'Croatia', 'Italy', 'Denmark', 'Switzerland', 'Serbia',
    'Japan', 'South Korea', 'Saudi Arabia', 'Iran', 'Australia', 'Qatar',
    'Morocco', 'Senegal', 'Nigeria', 'Egypt', 'Ghana', 'Cameroon',
    'Costa Rica', 'Panama',
    'Sweden', 'Poland', 'Austria', 'Norway', 'Chile', 'Paraguay',
    'Ivory Coast', 'Algeria', 'Tunisia', 'Mali', 'Iraq', 'Uzbekistan'
]

print(f"\n--- Configuración de la Simulación ---")
print(f"Número de equipos definidos para el Mundial 2026: {len(equipos_mundial_2026)}")
print("¡Todo listo para crear los grupos y comenzar a simular el torneo!")


--- Configuración de la Simulación ---
Número de equipos definidos para el Mundial 2026: 47
¡Todo listo para crear los grupos y comenzar a simular el torneo!


## **Crear función de predicción y simular un partido** ##

In [17]:
def predict_match(team1, team2, model, encoder):
    """
    Predice el resultado de un partido entre dos equipos.
    Devuelve las probabilidades: [Victoria Team2, Empate, Victoria Team1]
    """
    team1_encoded = encoder.transform([team1])
    team2_encoded = encoder.transform([team2])
   
    prediction = model.predict([team1_encoded, team2_encoded], verbose=0)
    return prediction[0]

team1 = 'Spain'
team2 = 'Morocco'
probabilities = predict_match(team1, team2, model, team_encoder)

print(f"\n--- Predicción para {team1} vs. {team2} ---")
print(f"Prob. Victoria {team1}: {probabilities[2] * 100:.2f}%")
print(f"Prob. Empate: {probabilities[1] * 100:.2f}%")
print(f"Prob. Victoria {team2}: {probabilities[0] * 100:.2f}%")


--- Predicción para Spain vs. Morocco ---
Prob. Victoria Spain: 67.69%
Prob. Empate: 23.25%
Prob. Victoria Morocco: 9.06%
